### Lab: Navigating the Lunar Lander with a Dueling Deep Q Network

### University of Virginia
### Reinforcement Learning
#### Last updated: February 27, 2024

---

#### Instructions:

You will work with the `LunarLander-v2` environment from `gymnasium` in this lab.  

An overview of the environment can be found [here](https://gymnasium.farama.org/).  
If you're curious about the source code, see [here](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/box2d/lunar_lander.py).

Your mission will be to implement a dueling deep Q -network using PyTorch.  
You might run this on Colab.

There are a few specific tasks outlined below for you to solve.

The bigger tasks will be to:

- Show that the algorithm works to train the agent in the environment
- Run episodes and show the results

**Submission**  
As you will likely have several files including this notebook, you can zip all files and submit.

---

![lunar](https://github.com/jackburke12/applied_reinforcement_learning/blob/main/07_deep_q_networks_extensions2/lunar_lander1.png?raw=1)

#### TOTAL POINTS: 12

---

**Hint:** Modules you may need to install include:

swig  
gym[box2d]  
gymnasium

#### 1) What is the penalty for crashing?  
**(POINTS: 1)**


-100 is the penalty for crashing.

#### 2) Set up the environment and run 2 steps by taking random actions.
**(POINTS: 1)**

In [10]:
pip install swig

  Using cached swig-4.4.0-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (3.5 kB)
Using cached swig-4.4.0-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.9 MB)


In [11]:
pip install "gymnasium[box2d]"

  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp312-cp312-linux_x86_64.whl size=2399001 sha256=8b2d2246d25a9869fbe45636904f39b20f5bf9bf9c820832b80f592225116e01
  Stored in directory: /root/.cache/pip/wheels/2a/e9/60/774da0bcd07f7dc7761a8590fa2d065e4069568e78dcdc3318
Successfully built box2d-py


In [12]:
import gymnasium as gym

env = gym.make("LunarLander-v3", render_mode="human")
obs, info = env.reset()
print("Initial Observation: ", obs)

for step in range(2):
    action = env.action_space.sample()
    next_obs, reward, terminated, truncated, info = env.step(action)
    print(f"Step {step+1}")
    print("Action:", action)
    print("Next obs:", next_obs)
    print("Reward:", reward, "Terminated:", terminated, "Truncated:", truncated)

Initial Observation:  [ 0.00727282  1.4180373   0.7366425   0.3163036  -0.00842058 -0.16686052
  0.          0.        ]
Step 1
Action: 1
Next obs: [ 0.01448488  1.4245894   0.72799367  0.29115334 -0.01512921 -0.13418403
  0.          0.        ]
Reward: 0.40069485419087525 Terminated: False Truncated: False
Step 2
Action: 0
Next obs: [ 0.02169723  1.4305431   0.72801507  0.2645212  -0.02183402 -0.13410841
  0.          0.        ]
Reward: -0.32746595659597233 Terminated: False Truncated: False


#### 3) Briefly discuss your approach to solving the problem  
**(POINTS: 2)**



The environment provides an 8-dimensional continuous state vector and 4 discrete actions: do nothing, fire left, fire main, and fire right. We'll create a dueling deep Q-network to land the module safely.

Our network will consist of an 8-feature input layer that takes the state vector as input. Two connected layers will be split into a value stream, V(s), and advantage stream, A(s,a). This is the core of the dueling network architecture. The streams will then combine to form Q(s,a), the state-action value function.

We will also use a replay buffer to store transitions, and sample from mini-batches for training. A target network will provide stable Q-targets and will be updated periodically from the online policy network.

The train step will compute the temporal-difference loss and update network parameters via gradient descent.

#### 4) Create supporting code files (`.py` format) to create the agent, train, and run episodes
**(POINTS: 6)**

Your code should include:

- **(POINTS: 1)** A class for the dueling DQN agent
- **(POINTS: 1)** An architecture with separate Value and Advantage streams
- **(POINTS: 1)** A method called `forward()` for the forward pass of the algorithm
- **(POINTS: 1)** A replay buffer
- **(POINTS: 1)** A training function
- **(POINTS: 1)** A function to run episodes

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from collections import deque

In [19]:
class DuelingDQN(nn.Module):
  def __init__(self, state_dim, action_dim):
    super(DuelingDQN, self).__init__()
    self.shared = nn.Sequential(
        nn.Linear(state_dim, 128),
        nn.ReLU(),
        nn.Linear(128,128),
        nn.ReLU()
    )

    self.value_stream = nn.Sequential(
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,1)
    )

    self.advantage_stream = nn.Sequential(
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64, action_dim)
    )

  def forward(self, state):
    x = self.shared(state)
    value = self.value_stream(x)
    advantage = self.advantage_stream(x)
    #combine value and advantage streams. An average of the advantage function is used, like the dueling q-network paper
    q_values = value + (advantage - advantage.mean(dim=1, keepdim = True))
    return q_values

class ReplayBuffer:
  def __init__(self, capacity):
    self.buffer = deque(maxlen=capacity)

  def push(self, state, action, reward, next_state, done):
    self.buffer.append((state, action, reward, next_state, done))

  def sample(self, batch_size):
    batch = random.sample(self.buffer, batch_size)
    states, actions, rewards, next_states, dones = map(np.array, zip(*batch))
    return (
        torch.FloatTensor(states),
        torch.LongTensor(actions),
        torch.FloatTensor(rewards),
        torch.FloatTensor(next_states),
        torch.FloatTensor(dones)
    )

  def __len__(self):
    return len(self.buffer)

def train_step(policy_net, target_net, optimizer, replay_buffer, batch_size, gamma):
  if len(replay_buffer) < batch_size:
    return None

  states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)

  q_values = policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)

  with torch.no_grad():
    max_next_q = target_net(next_states).max(1)[0]
    target = rewards + gamma * max_next_q * (1-dones)

  loss = F.mse_loss(q_values, target)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  return loss.item()

def run_episodes(
    env_name = "LunarLander-v3",
    num_episodes = 100,
    gamma = 0.99,
    lr = 1e-3,
    batch_size = 64,
    buffer_capacity = 100000,
    epsilon = 0.05,
    target_update_freq = 10
):

  env = gym.make(env_name)
  state_dim = env.observation_space.shape[0]
  action_dim = env.action_space.n

  policy_net = DuelingDQN(state_dim, action_dim)
  target_net = DuelingDQN(state_dim, action_dim)
  target_net.load_state_dict(policy_net.state_dict())
  target_net.eval()

  optimizer = torch.optim.Adam(policy_net.parameters(), lr=lr)
  replay_buffer = ReplayBuffer(buffer_capacity)

  rewards_history = []

  for episode in range(num_episodes):
    state, info = env.reset()
    total_reward = 0
    done = False

    while not done:
      if np.random.rand() < epsilon:
        action = env.action_space.sample()
      else:
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        q_values = policy_net(state_tensor)
        action = q_values.argmax().item()

      next_state, reward, terminated, truncated, info = env.step(action)
      done = terminated or truncated

      replay_buffer.push(state, action, reward, next_state, done)
      state = next_state
      total_reward += reward

      loss = train_step(policy_net, target_net, optimizer, replay_buffer, batch_size, gamma)

    if episode % target_update_freq == 0:
      target_net.load_state_dict(policy_net.state_dict())

    rewards_history.append(total_reward)
    print(f"Episode {episode+1}/{num_episodes} | Reward: {total_reward:.1f}")

  env.close()
  return rewards_history

#### 5) Run the training and show evidence that the agent is learning.  

For example, its average reward (score) should increase with more episodes.

**(POINTS: 1 if successful)**

In [20]:
rewards = run_episodes(num_episodes=500)

Episode 1/500 | Reward: -203.7
Episode 2/500 | Reward: -122.1
Episode 3/500 | Reward: -128.8
Episode 4/500 | Reward: -424.3
Episode 5/500 | Reward: -714.7
Episode 6/500 | Reward: -198.2
Episode 7/500 | Reward: -107.0
Episode 8/500 | Reward: -1.9
Episode 9/500 | Reward: -224.6
Episode 10/500 | Reward: -152.0
Episode 11/500 | Reward: -217.4
Episode 12/500 | Reward: -141.5
Episode 13/500 | Reward: -208.7
Episode 14/500 | Reward: -439.6
Episode 15/500 | Reward: -261.0
Episode 16/500 | Reward: -392.4
Episode 17/500 | Reward: -126.8
Episode 18/500 | Reward: -280.5
Episode 19/500 | Reward: -629.1
Episode 20/500 | Reward: 152.7
Episode 21/500 | Reward: -581.9
Episode 22/500 | Reward: -270.7
Episode 23/500 | Reward: -72.8
Episode 24/500 | Reward: -150.8
Episode 25/500 | Reward: -45.6
Episode 26/500 | Reward: -681.9
Episode 27/500 | Reward: -113.7
Episode 28/500 | Reward: -305.7
Episode 29/500 | Reward: -332.9
Episode 30/500 | Reward: -147.3
Episode 31/500 | Reward: -52.6
Episode 32/500 | Reward

#### 6) Run a few episodes and show results
**(POINTS: 1 if successful)**

In [18]:
import matplotlib.pyplot as plt

def moving_average(data, window_size=10):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

plt.figure(figsize=(10,6))
plt.plot(rewards, color='lightgray', label='Raw Reward')
plt.plot(moving_average(rewards, 10), color='blue', linewidth=2, label='Moving Average (10)')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('LunarLander-v3 — Smoothed Training Curve')
plt.legend()
plt.grid(True)
plt.show()